In [1]:
!pip install tensorflow_datasets
!pip install -U tensorflow-text

     |████████████████████████████████| 4.9 MB 10.7 MB/s 
     |████████████████████████████████| 462 kB 39.7 MB/s 


In [2]:
import collections
import logging
import os
import pathlib
import re
import string
import sys
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow as tf

In [3]:
logging.getLogger('tensorflow').setLevel(logging.ERROR)  # suppress warnings

In [4]:
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True,
                               as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incomplete3M3QQO/ted_hrlr_translate-train.tfrecord


...

  0%|          | 0/1193 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incomplete3M3QQO/ted_hrlr_translate-test.tfrecord


  0%|          | 0/1803 [00:00<?, ? examples/s]

Dataset ted_hrlr_translate downloaded and prepared to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0. Subsequent calls will reuse this data.


In [11]:
for pt_examples, en_examples in train_examples.batch(3).take(1):
  for pt in pt_examples.numpy():
    print(pt.decode('utf-8'))

  print()

  for en in en_examples.numpy():
    print(en.decode('utf-8'))

e quando melhoramos a procura , tiramos a única vantagem da impressão , que é a serendipidade .
mas e se estes fatores fossem ativos ?
mas eles não tinham a curiosidade de me testar .

and when you improve searchability , you actually take away the one advantage of print , which is serendipity .
but what if it were active ?
but they did n't test for curiosity .


In [12]:
model_name = "ted_hrlr_translate_pt_en_converter"
tf.keras.utils.get_file(
    f"{model_name}.zip",
    f"https://storage.googleapis.com/download.tensorflow.org/models/{model_name}.zip",
    cache_dir='.', cache_subdir='', extract=True
)

196608/184801 [===============================] - 0s 0us/step


'./ted_hrlr_translate_pt_en_converter.zip'

In [13]:
tokenizers = tf.saved_model.load(model_name)

In [14]:
[item for item in dir(tokenizers.en) if not item.startswith('_')]

['detokenize',
 'get_reserved_tokens',
 'get_vocab_path',
 'get_vocab_size',
 'lookup',
 'tokenize',
 'tokenizer',
 'vocab']

In [15]:
for en in en_examples.numpy():
  print(en.decode('utf-8'))

and when you improve searchability , you actually take away the one advantage of print , which is serendipity .
but what if it were active ?
but they did n't test for curiosity .


In [16]:
encoded = tokenizers.en.tokenize(en_examples)

for row in encoded.to_list():
  print(row)

[2, 72, 117, 79, 1259, 1491, 2362, 13, 79, 150, 184, 311, 71, 103, 2308, 74, 2679, 13, 148, 80, 55, 4840, 1434, 2423, 540, 15, 3]
[2, 87, 90, 107, 76, 129, 1852, 30, 3]
[2, 87, 83, 149, 50, 9, 56, 664, 85, 2512, 15, 3]


In [17]:
round_trip = tokenizers.en.detokenize(encoded)
for line in round_trip.numpy():
  print(line.decode('utf-8'))

and when you improve searchability , you actually take away the one advantage of print , which is serendipity .
but what if it were active ?
but they did n ' t test for curiosity .


In [18]:
tokens = tokenizers.en.lookup(encoded)
tokens

<tf.RaggedTensor [[b'[START]', b'and', b'when', b'you', b'improve', b'search', b'##ability',
  b',', b'you', b'actually', b'take', b'away', b'the', b'one', b'advantage',
  b'of', b'print', b',', b'which', b'is', b's', b'##ere', b'##nd', b'##ip',
  b'##ity', b'.', b'[END]']                                                 ,
 [b'[START]', b'but', b'what', b'if', b'it', b'were', b'active', b'?',
  b'[END]']                                                           ,
 [b'[START]', b'but', b'they', b'did', b'n', b"'", b't', b'test', b'for',
  b'curiosity', b'.', b'[END]']                                          ]>

In [19]:
def tokenize_pairs(pt, en):
    pt = tokenizers.pt.tokenize(pt)
    # Convert from ragged to dense, padding with zeros.
    pt = pt.to_tensor()

    en = tokenizers.en.tokenize(en)
    # Convert from ragged to dense, padding with zeros.
    en = en.to_tensor()
    return pt, en

In [20]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

In [21]:
def make_batches(ds):
  return (
      ds
      .cache()
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .map(tokenize_pairs, num_parallel_calls=tf.data.AUTOTUNE)
      .prefetch(tf.data.AUTOTUNE))


train_batches = make_batches(train_examples)
val_batches = make_batches(val_examples)

In [22]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

In [23]:
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

In [24]:
n, d = 2048, 512
pos_encoding = positional_encoding(n, d)
print(pos_encoding.shape)
pos_encoding = pos_encoding[0]

# Juggle the dimensions for the plot
pos_encoding = tf.reshape(pos_encoding, (n, d//2, 2))
pos_encoding = tf.transpose(pos_encoding, (2, 1, 0))
pos_encoding = tf.reshape(pos_encoding, (d, n))

(1, 2048, 512)


In [25]:
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [26]:
x = tf.constant([[7, 6, 0, 0, 1], [1, 2, 3, 0, 0], [0, 0, 0, 4, 5]])
create_padding_mask(x)

<tf.Tensor: shape=(3, 1, 1, 5), dtype=float32, numpy=
array([[[[0., 0., 1., 1., 0.]]],


       [[[0., 0., 0., 1., 1.]]],


       [[[1., 1., 1., 0., 0.]]]], dtype=float32)>

In [27]:
def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask  # (seq_len, seq_len)

In [28]:
x = tf.random.uniform((1, 3))
temp = create_look_ahead_mask(x.shape[1])
temp

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[0., 1., 1.],
       [0., 0., 1.],
       [0., 0., 0.]], dtype=float32)>

In [43]:
def scaled_dot_product_attention(q, k, v, mask):

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # step 1: calculating score w.r.t each word (query * key)

  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk) # step 2: divide scores by 8 (square root of dimension of key vector)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  # softmax add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # step 3: softmax score determines how much each word will be expressed at this position

  output = tf.matmul(attention_weights, v)  # step 4: multiply each value vector by the softmax score

  return output, attention_weights

In [31]:
def print_out(q, k, v):
  temp_out, temp_attn = scaled_dot_product_attention(
      q, k, v, None)
  print('Attention weights are:')
  print(temp_attn)
  print('Output is:')
  print(temp_out)

In [32]:
np.set_printoptions(suppress=True)

temp_k = tf.constant([[10, 0, 0],
                      [0, 10, 0],
                      [0, 0, 10],
                      [0, 0, 10]], dtype=tf.float32)  # (4, 3)

temp_v = tf.constant([[1, 0],
                      [10, 0],
                      [100, 5],
                      [1000, 6]], dtype=tf.float32)  # (4, 2)

# This `query` aligns with the second `key`,
# so the second `value` is returned.
temp_q = tf.constant([[0, 10, 0]], dtype=tf.float32)  # (1, 3)
print_out(temp_q, temp_k, temp_v)

Attention weights are:
tf.Tensor([[0. 1. 0. 0.]], shape=(1, 4), dtype=float32)
Output is:
tf.Tensor([[10.  0.]], shape=(1, 2), dtype=float32)


In [33]:
temp_q = tf.constant([[0, 0, 10],
                      [0, 10, 0],
                      [10, 10, 0]], dtype=tf.float32)  # (3, 3)
print_out(temp_q, temp_k, temp_v)

Attention weights are:
tf.Tensor(
[[0.  0.  0.5 0.5]
 [0.  1.  0.  0. ]
 [0.5 0.5 0.  0. ]], shape=(3, 4), dtype=float32)
Output is:
tf.Tensor(
[[550.    5.5]
 [ 10.    0. ]
 [  5.5   0. ]], shape=(3, 2), dtype=float32)


In [103]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads # choosing the query size

    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)

    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x, batch_size): # split the q,k,v
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0] # q matrix split across attention heads

    print("batch_size = ", q[0])

    # data split across multiple attention heads
    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)

    print("-----")
    print(q.shape)
    print("-----")
    print(q)
    print("*****")
    print("-----")
    print(k.shape)
    print("-----")
    print(k)
    print("*****")
    print("-----")
    print(v.shape)
    print("-----")
    print(v)
    print("*****")

    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
    
    print("After splitting according number of heads")
    print("-----")
    print(q.shape)
    print(q)
    print("-----")
    print(k.shape)
    print(k)
    print("-----")
    print(v.shape)
    print(v)
    print("-----")
    

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return output, attention_weights

In [104]:
temp_mha = MultiHeadAttention(d_model=6, num_heads=2)
y = tf.random.uniform((1, 4, 6))  # (batch_size, encoder_sequence, d_model)
out, attn = temp_mha(y, k=y, q=y, mask=None)
out.shape, attn.shape

batch_size =  tf.Tensor(
[[0.66553974 0.7304648  0.18866587 0.3504753  0.23087335 0.46859705]
 [0.85880125 0.53956187 0.12490427 0.9261582  0.34248567 0.17113769]
 [0.63041747 0.75447357 0.02376258 0.7317028  0.20341551 0.58052886]
 [0.22573304 0.43398893 0.6162751  0.0743407  0.5347661  0.23442936]], shape=(4, 6), dtype=float32)
-----
(1, 4, 6)
-----
tf.Tensor(
[[[-0.4015986   0.29809073  0.77032185 -0.5666553   0.8768784
   -0.75986296]
  [-0.38676536  0.09988675  1.07373    -0.33045843  0.620787
   -1.0516577 ]
  [-0.47312465  0.5635139   0.86715055 -0.30940968  0.6898178
   -1.0133907 ]
  [-0.24121553 -0.3605605   0.39568156 -0.5917467   0.917904
   -0.45290238]]], shape=(1, 4, 6), dtype=float32)
*****
-----
(1, 4, 6)
-----
tf.Tensor(
[[[-0.75520647  0.1213237   0.60672516  0.0366278   0.18505228
   -0.32366386]
  [-0.30993944  0.01496503  0.45335037 -0.3553778   0.13697234
   -0.30942416]
  [-0.74120736  0.30775508  0.7775059   0.10660562  0.21832101
   -0.478576  ]
  [-0.10255715

(TensorShape([1, 4, 6]), TensorShape([1, 2, 4, 4]))

In [105]:
print("output after multi-head attention")
print(out)
print(out.shape)
print("attention weights after multi-head attention")
print(attn)
print(attn.shape)

output after multi-head attention
tf.Tensor(
[[[-0.500578    0.3730204  -0.19112062  0.28125405 -1.2409353
   -0.75266397]
  [-0.5076131   0.37907422 -0.20189346  0.2764887  -1.241207
   -0.74624467]
  [-0.5066322   0.37860888 -0.20061445  0.2774839  -1.2428876
   -0.74771845]
  [-0.4888714   0.36020955 -0.17156412  0.28682256 -1.2346258
   -0.7623386 ]]], shape=(1, 4, 6), dtype=float32)
(1, 4, 6)
attention weights after multi-head attention
tf.Tensor(
[[[[0.27984357 0.2314751  0.310761   0.17792034]
   [0.2862632  0.23422223 0.3206683  0.15884624]
   [0.2816197  0.22308183 0.324696   0.1706025 ]
   [0.27177098 0.252157   0.27129728 0.20477474]]

  [[0.26069188 0.28743562 0.2773414  0.17453109]
   [0.26001984 0.27305424 0.2852567  0.18166928]
   [0.26157042 0.27293962 0.2866004  0.1788895 ]
   [0.26082146 0.28961703 0.26990402 0.17965737]]]], shape=(1, 2, 4, 4), dtype=float32)
(1, 2, 4, 4)


In [90]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

In [106]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

    return out2

In [109]:
sample_encoder_layer = EncoderLayer(6, 2, 2) # d_model, num_heads, dff

sample_encoder_layer_output = sample_encoder_layer(
    tf.random.uniform((1, 4, 6)), False, None)

sample_encoder_layer_output.shape  # (batch_size, input_seq_len, d_model)

batch_size =  tf.Tensor(
[[0.31055117 0.1928786  0.91100276 0.8948517  0.01817799 0.08618927]
 [0.044788   0.4582777  0.60476875 0.7813239  0.10253859 0.52800965]
 [0.49642193 0.18139255 0.43883252 0.5615456  0.82393825 0.87967145]
 [0.56004274 0.395882   0.8345238  0.02872694 0.86173713 0.40193808]], shape=(4, 6), dtype=float32)
-----
(1, 4, 6)
-----
tf.Tensor(
[[[ 0.2878205   0.32000095 -0.67756414 -0.72563195  0.133217
    0.3501251 ]
  [ 0.34952036  0.06438749 -0.5839757  -0.5714399  -0.0672904
    0.2763697 ]
  [-0.22334066  0.02346081 -0.48830968 -0.5797993   0.37586474
    0.40593567]
  [-0.5159214   0.31509775 -0.62984246 -0.24656168  0.70587224
   -0.30244407]]], shape=(1, 4, 6), dtype=float32)
*****
-----
(1, 4, 6)
-----
tf.Tensor(
[[[-1.1057508   0.74978805 -0.2679733   0.6096869  -0.5674712
   -1.0306933 ]
  [-1.1503931   0.38748616 -0.29360217  0.3967043  -0.21028067
   -1.2445905 ]
  [-0.95775473 -0.18162264  0.08224254  0.23248285 -0.2565841
   -0.884177  ]
  [-0.8091603

TensorShape([1, 4, 6])

In [42]:
print(sample_encoder_layer_output)

tf.Tensor(
[[[ 1.5039915   0.64581245 -0.7644472  ...  0.4336872   0.593974
    0.9076633 ]
  [ 2.1417646  -0.1954515  -0.38147455 ...  0.26239607  0.84514827
    0.14405678]
  [ 0.5392627  -0.89187443 -2.352561   ... -0.44159415  0.81794614
    0.55037683]
  ...
  [ 0.5260529   0.6966532  -1.3509872  ...  1.0121024  -0.6807412
    0.64469665]
  [ 1.2153506  -0.61581683 -1.1046088  ...  0.48606852 -0.03849971
   -0.31706414]
  [ 1.843132    0.5004921  -1.3465748  ... -0.44146803  0.8485915
    1.1139324 ]]

 [[ 1.0900512  -0.7661189  -1.8667824  ... -0.00638725 -0.36124092
   -0.2522634 ]
  [ 2.130491    0.8435218  -1.287596   ...  0.99923784 -0.5517835
    0.83475965]
  [ 1.0303661   0.9258312  -0.84796    ... -0.5535452   1.1477529
    0.8750177 ]
  ...
  [ 0.8908586   0.22615133 -2.270615   ... -0.2276543   0.07337688
    0.05146088]
  [ 1.6292783  -0.06275192 -1.5496111  ...  1.0023122   0.02692952
   -0.34952945]
  [ 1.9468089  -0.90692514 -0.8461679  ...  1.1516587  -0.20460992
 